# Clasificación de impagos de préstamos en un banco

Vamos a implementar un modelo para detectar impagos en los préstamos de un banco

## Índice

1. [Conexión a la base de datos](#mysql)
2. [Feature Extraction](#feature_extraction)
4. [Feature selection](#correlation)
5. [Transformation](#transformation)
6. [Modeling](#modeling)
7. [Feature Importance](#feature_importance)
8. [Próximos pasos](#future)

<a name='mysql'></a>
## 1. Conexión a la base de datos

In [ ]:
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
database_host = 'relational.fit.cvut.cz'
username = 'guest'
password = 'relational'
database_name = 'financial'

db = pymysql.connect(database_host,username,password,database_name)

<a name='feature_extraction'></a>
## 2. Extracción de variables

De las tablas **loan**, **account** y **district** extrae las siguientes variables en una única tabla
- identificador de la cuenta
- fecha del préstamo
- cantidad del préstamo
- duración del préstamo
- pagos mensuales
- estado del préstamo
- frecuencia de los extractos bancarios
- fecha de creación de la cuenta
- nº de habitantes del distrito
- variables del distrito

Transforma las fechas con `pandas.to_datetime`

Crea la variable `days_between` como la diferencia de días entre la fecha del préstamo y la fecha creación de la cuenta

Renombra las variables relacionadas con el distrito y crea la variable del ratio de crímenes por habitante

Crea el target binario

De la tabla **trans** obtén, para cada cuenta, la cantidad transferida y el balance

Cuenta el número de transacciones de cada cuenta

Calcula la media de cantidad y balance de las transacciones para cada cuenta

Une todos estos datos con el dataframe general

De la tabla **card**, añade el tipo de tarjeta para cada cuenta (solo titulares)

Une la tabla anterior a la general. Para aquellas cuentas sin tarjeta, ponle el valor "No"

De la tabla **client**, obtén la edad de los clientes (en el momento del préstamo), su sexo y añade una variable binaria que indique si el distrito del cliente coincide con el distrito de la cuenta

Une la tabla anterior con la general

### Tablón final

<a name='correlation'></a>
## 3. Selección de variables

Aplica técnicas para seleccionar las variables de entrada al modelo

<a name='transformation'></a>
## 4. Transformaciones

Estandariza las variables numéricas y convierte las categóricas mediante one-hot encoding

<a name='modeling'></a>
## 5. Modelado

Entrena uno o varios modelos, dividiendo el conjunto en train-test 

### Medición del desempeño

Utiliza las méticas de desempeño de los modelos de clasificación binarios, en el conjunto de train y test